In [ ]:
#verrijking van de harstilstand data doen eerst, dat is het allereerste. dan de slechte harstilstanden eruit halen, die dat al 
#volledig gedekt zijn door de ziekenhuizen, want die hoeven toch al niet meer gered te worden. Dan kan je zien welke AED's nuttig zijn en welke niet,
#door die netwerk analyse toe te passen en te zien welke AED's niet verbonden zijn (niet de dichtstbijzijnde) bij een harstilstand zijn. (PAS OP VOOR
#POTENTIELE RANDGEVALLEN, ALS ER BIJVOORBEELD meerdere harstilstanden vlak naast elkaar zijn, kan het zijn dat een AED eigenlijk slecht is maar toch het
#dichtst staat bij een harstilstand, die al gedekt is => HIER REKENING MEE HOUDEN, => OPLOSSING GEEN BIPARTITE GRAPH GEBRUIKEN, JE VERBINDT ALLE 
#HARTSTILSTANDEN ONDER EEN BEPAALDE TIJD MET ALLE AED'S DIE ONDER DIE BEPAALDE TIJD ZIJN, EN DAN ACHTERAF KAN JE BESLISSEN HOE HET ZIT)

In [ ]:
#AED's die te dicht bij de ziekenhuizen staan en daarom verwijderd worden, moeten ook gemarkeerd worden om die reden (reason for
#removal: too close to hospital). Dus pas op, je moet dat ook nog apart doen, als je enkel de harstilstanden verwijdert en niet ook de AED's,
#dan lijkt het alsof de AED's die te dicht stonden ook verwijderd werden omdat ze geen harstilstand coverden, terwijl dat eignelijk niet de reden was.

In [12]:
import pandas as pd
aed_locations = pd.read_csv('aed_locations_processed.csv')
card_arrest = pd.read_csv('card_arrest_processed.csv')
hospital_coordinates = pd.read_csv('hospital_coordinates_processed.csv')
removed_AED = pd.read_csv('removed_AED.csv')

In [4]:
#to calculate the distances, we use haversine formula, where we take the shortest distance between two points (a straight line).
#this is not always realistic (suppose an AED is placed on the corner of a building, then a straight line is impossible to take). Therefore we
#add 15% distance, to account for this.

,latitude,longitude
0,51.231381,4.415761
1,51.205787,4.412513
2,51.206556,4.470778
3,51.363865,4.985064
4,51.174354,4.420085
...,...,...
392,50.025633,5.359641
393,50.674803,5.633665
394,50.652888,5.578356
395,50.622523,5.636381


In [13]:
aed_locations

,id,address,number,postal_code,municipality,province
0,13.0,Blvd. Fr. Roosevelt,24.0,7060.0,SOIGNIES,Hainaut
1,70.0,Ch. De Wégimont,76.0,4630.0,Ayeneux,Liège
2,71.0,Place Saint - Lambert,NaN,4020.0,Liège,Liège
3,72.0,Rue du Doyard,NaN,4990.0,Lierneux,Liège
4,73.0,Fond Saint Servais,NaN,4000.0,Liège,Liège
...,...,...,...,...,...,...
14471,16660.0,Chaussée de Marche,799.0,5100.0,Wierde,Namur
14472,16661.0,Nekkerspoel-Borcht,19.0,2800.0,Mechelen,Antwerpen
14473,16664.0,Nieuwe Dreef,17.0,9160.0,Lokeren,Oost-Vlaanderen
14474,16665.0,Panterschipstraat,207.0,9000.0,Gent,Oost-Vlaanderen


In [14]:
card_arrest
#eigenlijk kan je gewoon de tijden nemen, de tijden tot interventie, en dan weet je al of de persoon gered is door een ziekenhuis. Je hoeft dan niet
#meer de afstand tov een ziekenhuis te berekenen, want zo ben je er al. Het probleem is dat er veel NA's zijn.

,eventType_trip,T0,T3,latitude_intervention,longitude_intervention,parsed_T0,parsed_T3,time_difference_seconds
0,P011 - Chest pain,2022-09-06 14:51:41.968677,NaN,50.87798,4.39229,2022-09-06 14:51:41.968677,NaN,NaN
1,P011 - Chest pain,2022-09-06 15:08:52.857352,2022-09-06 14:05:04.521346,50.89578,4.35817,2022-09-06 15:08:52.857352,2022-09-06 14:05:04.521346,-3828.336006
2,P011 - Chest pain,2022-09-06 16:36:15.795918,2022-09-06 14:50:46.344005,50.85592,4.42592,2022-09-06 16:36:15.795918,2022-09-06 14:50:46.344005,-6329.451913
3,P011 - Chest pain,2022-09-06 17:14:30.554547,2022-09-06 15:28:15.574171,50.80938,4.34743,2022-09-06 17:14:30.554547,2022-09-06 15:28:15.574171,-6374.980376
4,P039 - Cardiac problem (other than thoracic pain),2022-09-06 18:21:26.211055,NaN,50.86944,4.36080,2022-09-06 18:21:26.211055,NaN,NaN
...,...,...,...,...,...,...,...,...
37058,P011 - Chest pain,24MAY23:23:49:21,NaN,50.47360,4.84864,2023-05-24 23:49:21,NaN,NaN
37059,P011 - Chest pain,25MAY23:14:47:30,NaN,50.45880,4.88888,2023-05-25 14:47:30,NaN,NaN
37060,P003 - Cardiac arrest,25MAY23:18:04:00,NaN,49.97650,4.17427,2023-05-25 18:04:00,NaN,NaN
37061,P039 - Cardiac problem (other than thoracic pain),26MAY23:14:58:02,NaN,50.46170,4.62281,2023-05-26 14:58:02,NaN,NaN


In [15]:
#in a real world situation, we would if an AED would be useful for all three types of emergencies (P003, P011, P039), or
#only for the cardiac arrests (P003). We will asume that it is only useful for the cardiac arrests, thus we drop the other instances
card_arrest = card_arrest[card_arrest['eventType_trip'] == "P003 - Cardiac arrest"]
card_arrest

,eventType_trip,T0,T3,latitude_intervention,longitude_intervention,parsed_T0,parsed_T3,time_difference_seconds
9,P003 - Cardiac arrest,2022-09-06 20:31:42.666252,NaN,50.83808,4.30484,2022-09-06 20:31:42.666252,NaN,NaN
13,P003 - Cardiac arrest,2022-09-07 02:36:15.829707,2022-09-07 00:42:25.655123,50.82480,4.38185,2022-09-07 02:36:15.829707,2022-09-07 00:42:25.655123,-6830.174584
16,P003 - Cardiac arrest,2022-09-07 12:32:13.510586,2022-09-07 10:49:43.653950,50.83848,4.44441,2022-09-07 12:32:13.510586,2022-09-07 10:49:43.653950,-6149.856636
33,P003 - Cardiac arrest,2022-09-08 13:40:50.167814,2022-09-08 11:52:58.220448,50.84948,4.32034,2022-09-08 13:40:50.167814,2022-09-08 11:52:58.220448,-6471.947366
58,P003 - Cardiac arrest,2022-09-10 01:14:34.606094,2022-09-10 00:22:34.260161,50.84563,4.30821,2022-09-10 01:14:34.606094,2022-09-10 00:22:34.260161,-3120.345933
...,...,...,...,...,...,...,...,...
37011,P003 - Cardiac arrest,08APR23:08:31:39,NaN,50.45990,4.83448,2023-04-08 08:31:39,NaN,NaN
37025,P003 - Cardiac arrest,22APR23:14:53:28,NaN,50.37890,4.76960,2023-04-22 14:53:28,NaN,NaN
37029,P003 - Cardiac arrest,27APR23:10:43:33,NaN,50.26130,4.92232,2023-04-27 10:43:33,NaN,NaN
37032,P003 - Cardiac arrest,30APR23:14:31:51,NaN,50.59640,4.82970,2023-04-30 14:31:51,NaN,NaN


In [17]:
#a substantial amount of cardiac arrests does not contain the time needed for intervention.
na_count = card_arrest['time_difference_seconds'].isna().sum()
print(f"Number of NA values in 'time_difference_seconds': {na_count}")

Number of NA values in 'time_difference_seconds': 558


In [ ]:
#je kan de tijd nodig om een bepaalde afstand te bereiken berekenen door gebruik te maken van de data. Bereken de afstand
#tussen hospital en startlocatie -> maar startlocatie ontbreekt? Startlocatie is wel aanwezig, maar dan moet dat terug aangepast worden in de 
#preprocessing. en dat is wel echt aids. kan gewoon gemiddelde waarde nemen van tijd. (kijk ook naar de minimum waarde in tijd).

In [ ]:
#we estimate the time it would take to get from the hospital to the emergency location, based on distance and the value
#calculated above

In [29]:
#there are negative values present in the timings column. Even if asuming we can simply convert them to positive values, the 
#values are unrealistically high for practically all (more than an hour), so we consider these values as missing.

negative_values = card_arrest[card_arrest['time_difference_seconds'] < 0]['time_difference_seconds']

negative_count = negative_values.count()

negative_values_list = negative_values.tolist()

print(negative_values_list)
print(negative_count)

[-6830.174584, -6149.856636, -6471.947366, -3120.345933, -5111.783329, -6732.075481, -6293.029901, -6536.921347, -4503.50267, -6579.416427, -6758.248849, -6701.055291, -6081.778471, -6710.66801, -6570.838345, -6487.901567, -6169.761268, -6884.762954, -6044.905309, -6313.986671, -6706.130792, -3690.164941, -6443.617037, -5135.483311, -6687.304253, -6064.998433, -6039.626583, -6653.090254, -6697.922039, -6400.580312, -6878.940691, -6204.140027, -5038.370541, -6723.956466, -6911.891513, -6712.443673, -5879.896918, -6664.321175, -6639.903233, -6515.736195, -6825.039345, -6579.467391, -6750.402906, -3097.362677, -3048.313929, -3049.183563, -2785.019912, -2989.676244, -3176.302019, -3271.051081, -1605.42705, -2310.27625, -2664.001554, -3037.255123, -3013.62521, -2263.145241, -1874.957598, -2988.794495, -2979.404722, -3348.29185, -1692.692833, -2380.198291, -2599.22477, -3057.446272, -2884.561947, -2050.27464, -3036.963641, -3209.64753, -2184.90648, -3314.326655, -2836.714585, -3140.940256, -

In [30]:
#convert to absolute values first, otherwise the minimum values will be the impossible negative values.
card_arrest['time_difference_seconds'] = card_arrest['time_difference_seconds'].abs()

#considering the average time needed of 16 minutes and 49 seconds and the median value of 11 minutes 38 seconds, we consider the missing values as not
#reached in time by the ambulance.

min_value = card_arrest['time_difference_seconds'].min()
max_value = card_arrest['time_difference_seconds'].max()
median_value = card_arrest['time_difference_seconds'].median()
mean_value = card_arrest['time_difference_seconds'].mean()

# Print the results
print(f"Minimum value: {min_value}")
print(f"Maximum value: {max_value}")
print(f"Median value: {median_value}")
print(f"Average value: {mean_value}")


Minimum value: 79.255
Maximum value: 15499.091
Median value: 698.133
Average value: 1009.9325688690866


C:\Users\eloua\AppData\Local\Temp\ipykernel_7528\3063958062.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  card_arrest['time_difference_seconds'] = card_arrest['time_difference_seconds'].abs()


In [ ]:
#it is hard to find the exact time a person with cardiac arrest can survive without receiving aid. This exact value influences 
#the results of the analysis drastically however. In a real analysis the stakeholder would be contacted to provide an estimate of this time.
#we will asume that a person can go for 3 minutes without receiving aid.
na_count = card_arrest['time_difference_seconds'].isna().sum()